In [1]:
#Thứ tự chạy: 1

#Train model nhan dien cam xuc trong cau#

# -*- coding: utf-8 -*-
from __future__ import print_function
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import SGDClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from pyvi import ViTokenizer
import re
import string
import codecs

#Từ điển tích cực, tiêu cực, phủ định
path_nag = './sentiment/sentiment_dicts/nag.txt'
path_pos = './sentiment/sentiment_dicts/pos.txt'
path_not = './sentiment/sentiment_dicts/not.txt'

with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
    nag = f.readlines()
nag_list = [n.replace('\n', '') for n in nag]

with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
    pos = f.readlines()
pos_list = [n.replace('\n', '') for n in pos]
with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_ = f.readlines()
not_list = [n.replace('\n', '') for n in not_]


VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

def normalize_text(text):

    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": "nagative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "nagative", "😱": "nagative", "😸": "positive",
        "😾": "nagative", "🚫": "nagative",  "🤬": "nagative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
        '👎': ' nagative ', '😣': ' nagative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
        '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
        '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' nagative ', '😢': ' nagative ',
        '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' nagative ', '😊': ' positive ',
        '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' nagative ', '😭': ' nagative ',
        '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
        '^^': ' positive ', '😨': ' nagative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
        '😖': ' nagative ', '😀': ' positive ', ':((': ' nagative ', '😡': ' nagative ', '😠': ' nagative ',
        '😒': ' nagative ', '🙂': ' positive ', '😏': ' nagative ', '😝': ' positive ', '😄': ' positive ',
        '😙': ' positive ', '😤': ' nagative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
        '✌': ' positive ', '💕': ' positive ', '😞': ' nagative ', '😓': ' nagative ', '️🆗️': ' positive ',
        '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
        '💓': ' positive ', '😐': ' nagative ', ':3': ' positive ', '😫': ' nagative ', '😥': ' nagative ',
        '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
        '😗': ' positive ', '🤔': ' nagative ', '😑': ' nagative ', '🔥': ' nagative ', '🙏': ' nagative ',
        '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
        '😚': ' positive ', '❌': ' nagative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
        '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
        '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
        '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
        '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
        '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' nagative ',  '💀': ' nagative ',
        '😔': ' nagative ', '😧': ' nagative ', '😩': ' nagative ', '😰': ' nagative ', '😳': ' nagative ',
        '😵': ' nagative ', '😶': ' nagative ', '🙁': ' nagative ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' positive ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' nagative ',' cc ': ' nagative ','cute': u' dễ thương ','huhu': ' nagative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}

    for k, v in replace_list.items():
        text = text.replace(k, v)

    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
            if cp_text in pos_list:
                texts.append('positive')
            elif cp_text in nag_list:
                texts.append('nagative')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

#print(normalize_text('Nma kính đẹp chắc chắn 😎'))

class DataSource(object):

    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r', encoding='UTF-8') as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)
        b.append(a)

        return b[1:]

    def _create_row(self, sample, is_train=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')

            d['label'] = int(sample[-1].replace('\n', ' '))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', ' ')
                review = review.replace('.', ' ')


        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst

    def transform_to_dataset(self, x_set,y_set):
        X, y = [], []
        for document, topic in zip(list(x_set), list(y_set)):
            document = normalize_text(document)
            X.append(document.strip())
            y.append(topic)
            #Augmentation bằng cách remove dấu tiếng Việt
            X.append(no_marks(document))
            y.append(topic)
        return X, y

ds = DataSource()
train_data = pd.DataFrame(ds.load_data('./sentiment/data_clean/train.crash'))
new_data = []

#Thêm mẫu bằng cách lấy trong từ điển Sentiment (nag/pos)
for index,row in enumerate(nag_list):
    new_data.append(['pos'+str(index),'0',row])
for index,row in enumerate(nag_list):
    new_data.append(['nag'+str(index),'1',row])

new_data = pd.DataFrame(new_data,columns=list(['id','label','review']))
train_data.append(new_data)
test_data = pd.DataFrame(ds.load_data('./sentiment/data_clean/test.crash', is_train=False))

#Try some models
classifiers = [
            # MultinomialNB(),
            # DecisionTreeClassifier(),
            # LogisticRegression(),
            # SGDClassifier(),
            LinearSVC(fit_intercept = True,multi_class='crammer_singer', C=1),
        ]

X_train, X_test, y_train, y_test = train_test_split(train_data.review, train_data.label, test_size=0.3,random_state=42)
X_train, y_train = ds.transform_to_dataset(X_train,y_train)
X_test, y_test = ds.transform_to_dataset(X_test, y_test)

#THÊM STOPWORD LÀ NHỮNG TỪ KÉM QUAN TRỌNG
stop_ws = (u'rằng',u'thì',u'là',u'mà')

for classifier in classifiers:
    steps = []
    steps.append(('CountVectorizer', CountVectorizer(ngram_range=(1,5),stop_words=stop_ws,max_df=0.5, min_df=5)))
    steps.append(('tfidf', TfidfTransformer(use_idf=False, sublinear_tf = True,norm='l2',smooth_idf=True)))
    steps.append(('classifier', classifier))
    clf = Pipeline(steps)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    report1 = metrics.classification_report(y_test, y_pred, labels=[1,0], digits=3)

X_train, y_train = ds.transform_to_dataset(train_data.review, train_data.label)


#TRAIN OVERFITTING/ERRO ANALYSIS
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
report2 = metrics.classification_report(y_train, y_pred, labels=[1,0], digits=3)

#ERRO ANALYSIS
#for id,x, y1, y2 in zip(train_data.id, X_train, y_train, y_pred):
    #if y1 != y2:
        # CHECK EACH WRONG SAMPLE POSSITIVE/NAGATIVE
        #if y1!=1:#0:
            #print(id,x, y1, y2)

#CROSS VALIDATION
cross_score = cross_val_score(clf, X_train,y_train, cv=5)

#REPORT
print('DATASET LEN %d'%(len(X_train)))
print('TRAIN 70/30 \n\n',report1)
print('TRAIN OVERFITING\n\n',report2)
print("CROSSVALIDATION 5 FOLDS: %0.4f (+/- %0.4f)" % (cross_score.mean(), cross_score.std() * 2))

# #SAVE FILE SUBMIT
#test_list = []
#for document in test_data.review:
#    document = normalize_text(document)
#    test_list.append(document)
#print(test_list)
#y_predict = clf.predict(test_list)
#test_data['label'] = y_predict
# test_data['content'] = test_list
#test_data = test_data.sort_values(by=['label'])
#test_data[['id', 'label']].to_csv('submit.csv', index=False)


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\base.py:928: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\base.py:928: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


DATASET LEN 32146
TRAIN 70/30 

               precision    recall  f1-score   support

           1      0.938     0.945     0.941      4438
           0      0.952     0.947     0.950      5206

    accuracy                          0.946      9644
   macro avg      0.945     0.946     0.946      9644
weighted avg      0.946     0.946     0.946      9644

TRAIN OVERFITING

               precision    recall  f1-score   support

           1      0.981     0.985     0.983     14766
           0      0.987     0.984     0.985     17380

    accuracy                          0.984     32146
   macro avg      0.984     0.984     0.984     32146
weighted avg      0.984     0.984     0.984     32146

CROSSVALIDATION 5 FOLDS: 0.9436 (+/- 0.0096)


In [2]:
#Thứ tự chạy: 2
#Phần lọc itemID và shopID ra từ link
def getID(URL):
    group = ''
    itemID = shopID = ''
    chk = ''
    cnt = 0
    isdone = False
    for i in URL:
        if chk != 'i.':
            if i != '-':
                group += i
                if group == 'i.':
                    chk = group
            else: group = ''
        else:
            if isdone == False:
                if i != '.':
                    shopID += i
                else: isdone = True
            else:
                if i in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                    itemID += i
                else: return itemID, shopID
    return itemID, shopID
print(getID('https://shopee.vn/Kính-cận-đổi-màu-mắt-kính-nam-cận-râm-2-in1-mắt-cận-đổi-màu-theo-ánh-nắng-i.18722862.5560946376?adsid=0&campaignid=0&position=1'))

('5560946376', '18722862')


In [16]:
#Thứ tự chạy: 3

#Lấy các review từ itemID và shopID đã có ở trên thông qua API Shopee
import requests, time
from IPython.display import display, clear_output


headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"
}

if (int(input('Chọn 1 để nhập id, chọn 2 để nhập URL:')) == 1):
    itemID = input('Nhập id item:')
    shopID = input('Nhập id shop:')
else:
    productURL = input('Nhap URL san pham:')
#itemID = input('Nhap id cua item:')
#shopID = input('Nhap id cua shop:')

    itemID, shopID = getID(productURL)


path_not = './sentiment/sentiment_dicts/not.txt'
with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_ = f.readlines()
not_list = [n.replace('\n', '') for n in not_]
api = "https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&type=0&limit=6"
api += '&itemid=' + itemID + '&shopid=' + shopID
r = requests.get(api).json()
api = "https://shopee.vn/api/v2/item/get?" + 'itemid=' + itemID + '&shopid=' + shopID
#print(api)
r1 = requests.get(api, headers=headers).json()
#print(r1)
tensp = r1['item']['name']
rating_star = 0


rv_num = r['data']['item_rating_summary']['rating_total']
rcount_text = r['data']['item_rating_summary']['rcount_with_context']
data = []
#data.append({'ID san pham:': itemID, 'ID shop:': shopID, 'Tong danh gia:': rv_num})
offset_length = 0
i = 1
chk_empty = 0
start = time.time()
estimate_start = time.time()
estimate_time = 0
time_step = 0.5
rv_count = 1
rv_left = rcount_text
while i < rcount_text:
    if (time.time() - start > 6000):
        break
    if (time.time() - estimate_start > time_step):
        estimate_time = rv_left/(rv_count/time_step)
        rv_left -= rv_count
        estimate_start = time.time()
        rv_count = 1
    api = "https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&type=0&limit=6"
    api += '&itemid=' + itemID + '&shopid=' + shopID + '&offset=' + str(offset_length)
    r = requests.get(api).json()
    if r['data']['ratings'] == []:
        print(r)
        break
    #print(r['data']['ratings'])
    clear_output(wait=True)
    print('Ước tính thời gian còn lại:', estimate_time//60, 'phút', estimate_time%60,'giây')
    print('Load review: ', i, ' offset:', offset_length)
    for departure in r['data']['ratings']:
        if departure['comment'] == '':
            chk_empty += 1
            continue
        dem_tu = 0
        for chu in departure['comment']:
            if chu == ' ':
                dem_tu += 1
            if dem_tu >= 13:
                break
        if dem_tu < 13 or dem_tu >500: 
            offset_length += 1
            continue
        data.append({
        'Number': i,
        'Nguoi dung': departure['author_username'],
        'Nhan xet': departure['comment'],
        'Sao': departure['rating_star']
        })
        #print(estimate_time)
        #print("\nNumber: ", data[i-1]['Number'])
        #print("Nguoi dung: ", data[i-1]['Nguoi dung'])
        #print("Nhan xet: ", data[i-1]['Nhan xet'])
        #print("Sao: ", data[i-1]['Sao'])
        #print("-----------------")

        i += 1
        rv_count += 1

        offset_length += 1
    if chk_empty > 5:
        #i += 1
        offset_length +=1
        chk_empty = 0
    #rv_num -= 6
clear_output()
up = 1
for x in r['data']['item_rating_summary']['rating_count']:
    rating_star += up * x
    up += 1
rating_star /= r['data']['item_rating_summary']['rating_total']
print('Thời gian chạy:', (time.time() - start)//60, 'phút', (time.time() - start)%60,'giây')
print('Sao trung bình:', rating_star)
print('Tên sản phẩm:', tensp)
print('Load review:', i)
print("ID san pham:", itemID, "\nID shop:", shopID, "\nTong danh gia:", rv_num, "\nTong danh gia co binh luan:", rcount_text)


Thời gian chạy: 0.0 phút 51.61233448982239 giây
Sao trung bình: 4.879432624113475
Tên sản phẩm: Apple iPad Air 4 10.9 inch Wi-Fi
Load review: 150
ID san pham: 5076952578 
ID shop: 88201679 
Tong danh gia: 423 
Tong danh gia co binh luan: 231


In [17]:
import unidecode

def SentenceSpliter(sentence):
    split = []
    word = ''
    for i in range(0, len(sentence)):
        if sentence[i] == ' ':
            split.append(word)
            word = ''
        elif i == len(sentence) - 1 and word != '':
            word += sentence[i]
            split.append(word)
        else: word += sentence[i]
    return split

def SentenceCombiner(split):
    sentence = ''
    for i in range(0, len(split)):
        if i != len(split)-1:
            sentence += split[i] + ' '
        else:
            sentence += split[i]
    return sentence

def XoaDau(word, syllable):
    rmv_accent = word
    for x in rmv_accent:
        for s in syllable:
            if x != s:
                #print(syllable[s])
                for j in syllable[s]:
                    if x == j:
                        rmv_accent = rmv_accent.replace(x, s)
    return rmv_accent

def VanNguyenAm(word, syllable):
    #Vần đơn giản 1: chỉ chứa 1 nguyên âm
    correct = False
    if len(word) == 1:
            while (correct != True):
                for x in syllable:
                    if x in ['e', 'a', 'o', 'ê', 'ơ', 'ô', 'i', 'y', 'ư', 'u']:
                        if word != x:
                            for y in range(0, 5):
                                if word == syllable[x][y]:
                                    correct = True
                        else:
                            correct = True
                    else: continue
                break
    return correct

#Vần đơn giản (2): Nguyên âm + phụ âm cuối ~ Vần ngược
def VanDonGian2(word, syllable, vowel, consonant):
    bangVanLoai2 = [[1, 1, 1, 1, 0, 1, 1, 1],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [1, 0, 0, 1, 1, 0, 0, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1],
                    [0, 1, 1, 1, 0, 1, 1, 1],
                    [0, 0, 1, 1, 0, 0, 1, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1],
                    [1, 0, 1, 1, 1, 0, 1, 1]]
    correct = [False, False]
    for j in syllable:
        if word[0] == j:
            x = vowel['singleVowel'].index(word[0])
            correct[0] = True
            break
        else:
            for t in range(0, 5):
                if word[0] == syllable[j][t]:
                    x = vowel['singleVowel'].index(j)
                    correct[0] = True
                    break
    if correct[0] == True:
        if word[1:] in consonant['tConsonant']:
            y = consonant['tConsonant'].index(word[1:])
            correct[1] = True
        if correct[1] == True:
            if bangVanLoai2[x][y] != 1:
                return False
            return True
    return False

#Kiểm tra vần hòa âm và hợp âm
def KtraHoaAmHopAm(word, syllable, consonant):
    rmv_accent = word
    tailConsonant = ''
    chk = ''
    for i in range(1,3):
        chk += rmv_accent[i-3]
        if chk in consonant['tConsonant']:
            tailConsonant = chk
            #rmv_accent.replace(rmv_accent[-1], '')
        else: chk = ''
    #print(tailConsonant)
    for i in range(len(tailConsonant)):
        rmv_accent = rmv_accent.rstrip(rmv_accent[-1])
    #print(rmv_accent)
    if rmv_accent in ['eo', 'ao', 'ai', 'oe', 'oa', 'oi', 'êu', 'ơi', 'ôi', 'ia',
    'iu', 'ưa', 'ưi', 'ưu', 'ua', 'uê', 'ui', 'uy']:
        return True

    if rmv_accent in ['ay', 'ây', 'au', 'âu']:
        if tailConsonant == '':
            return True
        return False
    if rmv_accent in ['oă', 'uâ', 'iê', 'yê', 'ươ', 'uô']:
        if tailConsonant != '':
            return True
    if rmv_accent in ['uôi', 'ươi', 'ươu','yêu', 'iêu']:
        return True
    return False

def SentenceCorrector(splits):
    #split có dạng: split = ['từ', 'ví, 'dụ'...]
    #singleVowel: Nguyên âm đơn -> Có thể đứng 1 mình hoặc ghép với những chữ khác
    #sVowel: Bán nguyên âm -> Không thể đứng 1 mình hay đứng cuối mà phải có phụ âm cuối đi theo
    #hConsonant: Phụ âm đầu: đứng trước nguyên âm
    #tConsonant: Phụ âm cuối: đứng sau nguyên âm
    
    vowel = {'singleVowel':['i', 'y', 'ư', 'u', 'ê', 'ơ', 'â', 'ô', 'e', 'a', 'ă', 'o'],
    'sVowel':['ă', 'â', 'iê', 'yê', 'uô', 'ươ']}
    consonant = {'hConsonant':['b', 'c', 'd', 'đ', 'g', 'gh', 'h', 'k', 'l', 'm',
    'n', 'r', 's', 't', 'v', 'x', 'ch' , 'gi', 'kh', 'nh', 'ng', 'ngh', 'qu', 'ph',
    'th', 'tr'], 
    'tConsonant': ['c', 'ch', 'm', 'n', 'ng', 'nh', 'p', 't']}
    syllable = {'i':['í', 'ì', 'ỉ', 'ĩ', 'ị'], 'y': ['ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ'],
    'ư': ['ứ', 'ừ', 'ử', 'ữ', 'ự'], 'u': ['ú', 'ù', 'ủ', 'ũ', 'ụ'],
    'ê': ['ế', 'ề', 'ể', 'ễ', 'ệ'], 'ơ': ['ớ', 'ờ', 'ở', 'ỡ', 'ợ'],
    'â': ['ấ', 'ầ', 'ẩ', 'ẫ', 'ậ'], 'ô': ['ố', 'ồ', 'ổ', 'ỗ', 'ộ'],
    'e': ['é', 'è', 'ẻ', 'ẽ', 'ẹ'], 'a': ['á', 'à', 'ả', 'ã', 'ạ'],
    'ă': ['ắ', 'ằ', 'ẳ', 'ẵ', 'ặ'], 'o': ['ó', 'ò', 'ỏ', 'õ', 'ọ']}
    rmv = -1

    split = splits.copy()
    for i in splits:
        rmv +=1
    #Ktra điều kiện: Từ = vần Nguyên Âm:
        if (len(i)==1):
            #print('1 tu:', i)
            if VanNguyenAm(i, syllable) != True:
                split.pop(rmv)
                rmv -= 1
        else:
            #Tách phụ âm đầu
            #pad = True
            headConsonant = ''
            chk = ''
            cnt = 0
            for y in i:
                cnt += 1
                chk += y
                if chk in consonant['hConsonant']:
                    headConsonant = chk
                else:
                    if cnt > 1: break
                #if headConsonant == '': 
                #    split.pop(rmv)
                #    pad = False
                #    break
            #if pad == False:
            #    continue
            #print('tách phụ âm đầu: ', headConsonant)
            i = i.replace(headConsonant, '')
            i = XoaDau(i, syllable)
            #print(i)
            if (len(i)<1):
                split.pop(rmv)
                rmv -=1
                continue
            if (len(i) == 1):
                #print('TH1:', i)
                if VanNguyenAm(i, syllable) == True:
                    continue
                split.pop(rmv)
                rmv -= 1
                continue
            if VanDonGian2(i, syllable, vowel, consonant) == True:
                #print('TH2:', i)
                continue
            elif KtraHoaAmHopAm(i, syllable, consonant) == True:
                #print('TH3:', i)
                continue
            else: 
                #print('TH5:', i)
                split.pop(rmv)
                rmv -= 1
    return split




In [34]:
sentence = "phân loại hàng silver ipad air 4 có kiểu dáng mới tương tự iPad Pro 2020 nhưng có kích thước nhỏ hơn và dày chỉ 6.1 mm, trọng lượng đạt 460 g dễ dàng mang theo bên mình mọi lúc mọi nơi thiết kế này giúp tương thích với bàn phím Apple Smart Keyboard Folio Magic Keyboard của iPad Pro 11 inch và hỗ trợ bút Apple Pencil 2"
print(SentenceSpliter(sentence))
print(SentenceCombiner(SentenceCorrector(SentenceSpliter(sentence))))



['phân', 'loại', 'hàng', 'silver', 'ipad', 'air', '4', 'có', 'kiểu', 'dáng', 'mới', 'tương', 'tự', 'iPad', 'Pro', '2020', 'nhưng', 'có', 'kích', 'thước', 'nhỏ', 'hơn', 'và', 'dày', 'chỉ', '6.1', 'mm,', 'trọng', 'lượng', 'đạt', '460', 'g', 'dễ', 'dàng', 'mang', 'theo', 'bên', 'mình', 'mọi', 'lúc', 'mọi', 'nơi', 'thiết', 'kế', 'này', 'giúp', 'tương', 'thích', 'với', 'bàn', 'phím', 'Apple', 'Smart', 'Keyboard', 'Folio', 'Magic', 'Keyboard', 'của', 'iPad', 'Pro', '11', 'inch', 'và', 'hỗ', 'trợ', 'bút', 'Apple', 'Pencil']
phân hàng có kiểu dáng mới tương tự nhưng có kích thước nhỏ hơn và dày chỉ trọng lượng đạt dễ dàng mang theo bên mình mọi lúc mọi nơi thiết kế này giúp tương thích với bàn phím của và hỗ trợ bút


In [18]:
#Thứ tự chạy: 4

import re
import string

#Làm sạch dữ liệu review
def normalize_text(texts):
    
    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Chuẩn hóa 1 số sentiment words/English words
        ':)) ': ' hài lòng ', ' shiper ': ' người_giao_hàng ', ' shipper ': ' người_giao_hàng ', ' giao hàng ': ' giao_hàng ' , ' mng ': ' mọi người ', ' tnao ': ' thế nào ' , ' m.n ': ' mọi người' , ' tr ': ' trời ', ' nma ': ' nhưng mà ', ' mn ': ' mọi người ', ' mk ': ' mình ' , ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' thích ','hehe': ' thích ','hihi': ' thích ', 'haha': ' thích ', 'hjhj': ' thích ',
        ' lol ': ' tệ ',' cc ': ' tệ ','cute': u' dễ thương ','huhu': ' tệ ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' thích ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' thích ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}
    
    sents = []
    cau = '';
    count = 0
    for text in texts:
        count += 1
        #print(text['Nhan xet'])
        cau = text['Nhan xet']
        #cau = normalize_text(cau)
        sao = text['Sao']
        #Sửa các icon, từ viết tắt
        if cau == None:
            cau = ''
        for k, v in replace_list.items():
                cau = cau.replace(k, v)
        #print(cau)
        #Remove các ký tự kéo dài: vd: đẹppppppp
        texts = cau.split()
        len_text = len(texts)
        for i in range(len_text):
            cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        #else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
            #if cp_text in pos_list:
            #    texts.append('')
            #if cp_text in nag_list:
            #    texts.append('nagative')
        
        cau = u' '.join(texts)
        cau = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), cau, flags=re.IGNORECASE)
        preinput = re.split('(?<!^)\s*[.\n]+\s*(?!$)', cau)
        pre = dict()
        
        #Xóa kí tự đặc biệt
        for i in preinput:
            pre['Nhan xet'] = re.sub(r'[^a-zA-ZÀ-ỹ\s]', '', i, re.I|re.A).lower()
            pre['Nhan xet'] = SentenceCombiner(SentenceCorrector(SentenceSpliter(i)))
            pre['Sao'] = sao
        sents.append(pre)
        clear_output(wait=True)
        print('Review', count , ':', pre['Nhan xet'])
        print('.\n.\n.')
    return sents

def pre_process(raw_data):
    data = normalize_text(raw_data)
    #data.pop()
    return data

#Bước chuẩn hóa dữ liệu
#for r in range(len(seg)):
#    seg[r] = normalize_text(seg[r])
seg = pre_process(data)
#print(seg[:5]) #Dòng này để xem các review đã qua tiền xử lý
print('Đã xử lý xong các review.')


Review 149 : quá cửa hàng đóng gói kỹ có điều giao hơi nhiệt dễ
.
.
.
Đã xử lý xong các review.


In [20]:
#Thứ tự chạy: 5
#Tokenize các từ trong review
from pyvi import ViTokenizer
segs = []
cnt = 1
for token in seg:
    sub_segs = []
    clear_output(wait=True)
    #print('Xử lý review:', cnt, ':', token['Nhan xet'])
    cnt += 1
    doc = ViTokenizer.tokenize(str(token['Nhan xet']))
    #print(pos_tag(str(sub))
    sub_segs.append(str(doc))
    segs.append(sub_segs)
#print(segs[:5])
print('Hoàn thành tokenize.')

Hoàn thành tokenize.


In [21]:
#Thứ tự chạy: 6
#Chứa các đánh giá đã được phân lớp
review_class = []


#Tách câu:
def sentence_segment_original(text):
    sents = re.split('(?<!^)\s*[.\n]+\s*(?!$)', text)
    return sents

#Xử lý
for rv in segs:
    singleRVClass = []
    y_pred = clf.predict(rv)
    for i in y_pred:
        singleRVClass.append(i)
    review_class.append(singleRVClass)
#print(review_class)

#Tinh tong % tot / xau tren tat ca review
def Calculate_Review_Score(pred):
    score = []
    for i in pred:
        sum = 0
        for j in i:
            if j == 0:
                sum += 1
        score.append(sum/len(i))
    #print(score)
    for i in score:
        sum += i
    sum /= len(score)
    return(sum)
    #if sum < 0.3:
    #    return 'Sản phẩm được đánh giá rất tệ'
    #elif sum < 0.4:
    #    return 'Sản phẩm được đánh giá tệ'
    #elif sum <= 0.5:
    #    return 'Sản phẩm được đánh giá khá tệ'
    #elif sum < 0.7:
    #    return 'Sản phẩm được đánh giá khá tốt'
    #elif sum < 0.85:
    #    return 'Sản phẩm được đánh giá tốt'
    #else:
    #    return 'Sản phẩm được đánh giá rất tốt'

Product_Score = Calculate_Review_Score(review_class)
if Product_Score <= 0.5:
    good_bad = 1
else: good_bad = 0
print('Điểm đánh giá sản phẩm dựa trên review (thang điểm 10): ', Product_Score*10)


Điểm đánh giá sản phẩm dựa trên review (thang điểm 10):  6.308724832214764


In [22]:
#Thứ tự chạy: 7

### SECTION LOAI BO STOPWORDS ###
from underthesea import pos_tag
import pandas as pd

#Thêm dữ liệu từ file
file = open('./data/vietnamese_stopwords.txt','r', encoding = "utf8")
stopwords = file.read()
df = pd.read_csv('./data/600_adj_pairs.txt', delimiter="\t", sep=" ", encoding = "utf8", index_col="Relation")
df.columns = ['Word1', 'Word2']
SYN = [] #Danh sách từ đồng nghĩa
#print(df)
SYN = df.loc["SYN"]
#print(SYN['Word1'])


segg = segs.copy()
#print(stopwords)
stopword = re.split('(?<!^)\s*[.\n]+\s*(?!$)', stopwords)
i = x =0

#Thay thế khoảng trống của từ trong bộ stopword bằng ký tự '_' để dễ xử lý hơn
#stopwords = []
for word in stopword:
    stopword[x] = word.replace(' ', '_')
    x+=1
    #print(word)


### LOAI BO STOPWORDS ###
segss = []
for a in segg:
    tempseg = []
    for b in a:
        tempb = ''
        for c in b:
            #print(b)
            if c == ' ':
                #print(tempb)
                if tempb not in stopword and tempb != '':
                    tempseg.append(tempb)
                if tempb == 'tốt': print(tempseg)
                tempb = ''
            else:
                tempb += c
        #tempseg.append(tempb)
    if tempseg != []:
        segss.append(tempseg)
    i += 1
#print(segss[:5])

#Duyệt tất cả các tính từ vào bảng TABLE_ADJ
TABLE_ADJ = []
KiemTra = []
Count_All_Adjs = 0
count = 0
for i in segss:
    adj = dict()
    cau = ''
    for b in i:
        cau += b + ' '
    a = pos_tag(cau)
    count += 1
    #print(a[0][1])
    for b in pos_tag(cau):
        if b[1] == 'A':
            if b[0] not in KiemTra:
                adj['WORD'] = b[0]
                KiemTra.append(b[0])
                clear_output(wait=True)
                print('Xét câu', count, ':', cau)
                print('Loại từ: ', b[1], ', Từ: ', b[0])
                #print(KiemTra)
                print('Số lượng tính từ đã xét:', Count_All_Adjs)
                print('Số lượng tính từ khác nhau trong bảng: ', len(TABLE_ADJ))
                similar_words_1 = SYN.loc[SYN['Word1'] == b[0]]
                similar_words_2 = SYN.loc[SYN['Word2'] == b[0]]
                adj['COUNT'] = 1
                if similar_words_1.empty != True:
                    adj['SIMILAR'] = similar_words_1['Word2'].unique()[0]
                    adj['COUNT'] += 1
                    KiemTra.append(adj['SIMILAR'])
            
                if similar_words_2.empty != True:
                    adj['SIMILAR'] = similar_words_2['Word1'].unique()[0]
                    adj['COUNT'] += 1
                    KiemTra.append(adj['SIMILAR'])
                    
                else:
                    adj['SIMILAR'] = ''
                if adj not in TABLE_ADJ:
                    TABLE_ADJ.append(adj)
                    #print(TABLE_ADJ)
            else:
                for x in TABLE_ADJ:
                    if x['WORD'] == b[0]:
                        x['COUNT'] += 1
                    elif x['SIMILAR'] == b[0]:
                        x['COUNT'] += 1
            Count_All_Adjs += 1
print('Số lượng tính từ đã xét xong: ', Count_All_Adjs)
TABLE_ADJ.append(Count_All_Adjs)
#print(TABLE_ADJ)
#print(TABLE_ADJ[:5])
#print(segss[:4])

Xét câu 116 : trợ gửi hoá_đơn mua hàng siêu đọc nhắn thèm phản phục_vụ kém trả_lời 
Loại từ:  A , Từ:  kém
Số lượng tính từ đã xét: 80
Số lượng tính từ khác nhau trong bảng:  19
Số lượng tính từ đã xét xong:  82


In [23]:
#Thứ tự chạy: 8

#Kiểm tra độ tin tưởng của từng review dựa trên bảng tính từ:
import numpy as np
def Review_score(TatCaReviewDaXuLy, TABLE_ADJ, review, index, rating_star):
    score = 0
    constrain_2tt = 0
    constrain_2tt_trung = 0
    if abs(rating_star - TatCaReviewDaXuLy[index]['Sao']) <= 2:
        score += 0.2
    else: score += 0.1
    if clf.predict([review])[0] == good_bad:
        score += 0.1*good_bad
    else:
        score += 0.1*(1-good_bad)
    for word in pos_tag(review):
        if word[1] == 'N':
            constrain_2tt += 1
        if word[1] == 'A':
            constrain_2tt += 1
            if constrain_2tt == 2:
                score += 0.1
            for x in TABLE_ADJ[:-1]:
                if x['WORD'] == word[0]:
                    constrain_2tt_trung += 1
                    if constrain_2tt_trung == 2:
                        score += 0.1
                    score += x['COUNT']/TABLE_ADJ[-1]*0.5
    return score
mini = 1
maxi = 0
All_review_score = []
idx = 0
idx_min = idx_max = 0
cnt = 1
for i in segss:
    cau = ''
    for b in i:
        cau += b + ' '
        score = Review_score(seg, TABLE_ADJ, cau, idx, rating_star)
        if score > 1:
            score = 1
        if score < 0: score = 0
    if score <= mini: 
        mini = score
        idx_min = cnt - 1
    if score >= maxi and score < 0.9: 
        maxi = score
        idx_max = cnt - 1
    clear_output(wait=True)
    print('Sao trung bình:', rating_star)
    print('Tính điểm cho review', cnt, ':', score)
    print('Điểm thấp nhất:', mini, '\nReview:', data[idx_min]['Nhan xet'], '\nSao:', data[idx_min]['Sao'], '\nĐiểm cao nhất:', maxi, '\nReview:', data[idx_max]['Nhan xet'], '\nSao:', data[idx_max]['Sao'])
    cnt += 1
    All_review_score.append(score)
    idx += 1
print(All_review_score[:30])
#print(All_review_score[int(input('Nhập thứ tự câu:'))])
np_review = np.array(All_review_score)
#print(All_review_score)
#print(seg[:6])
idx = np.argpartition(np_review, -4)[-4:]
#print(idx)
print('Các review có điểm cao nhất:')
for i in idx:
    print(data[i]['Nhan xet'], ' score: ', All_review_score[i])
    #print('Score:',All_review_score[i])


Sao trung bình: 4.879432624113475
Tính điểm cho review 121 : 0.42439024390243907
Điểm thấp nhất: 0.2 
Review: Sản phẩm tốt. Giao hàng nhanh! Mua đc flash sale nên giá khá rẻ! Sẽ ủng hộ các sản phẩm khác cho shop! 
Sao: 5 
Điểm cao nhất: 0.5158536585365854 
Review: bự hơn mình tưởng, xác nhận hơi lâu nhưng xứng đáng để chờ đợi nhe 
Sao: 5
[0.30000000000000004, 0.3243902439024391, 0.2, 0.2, 0.2, 0.31829268292682933, 0.31219512195121957, 0.30000000000000004, 0.30000000000000004, 0.20609756097560977, 0.24268292682926831, 0.30000000000000004, 0.30000000000000004, 0.42439024390243907, 0.31219512195121957, 0.2, 0.2, 0.2, 0.21829268292682927, 0.30000000000000004, 0.2304878048780488, 0.22439024390243903, 0.26707317073170733, 0.30000000000000004, 0.2, 0.31829268292682933, 0.44268292682926835, 0.39146341463414636, 0.2, 0.2]
Các review có điểm cao nhất:
Giao hàng siêu tốc chỉ sau 3 ngày không tính ngày đặt đã nhận được hàng dù đang khó khăn mùa dịch. Mọi thứ đều đẹp và nguyên vẹn.  score:  0.44268